In [1]:
import gurobipy as gp
import pandas as pd
from gurobipy import GRB

In [70]:
plants = ["Afyon", "Konya", "Import (Izmir)"]
br_exist = ["Istanbul", "Ankara"]
br_pot = ["Izmir", "Sakarya", "Adana"]
dc = ["Istanbul", "Izmir", "Antalya", "Bursa", "Kayseri", "Export (Izmir)"]  # Distribution Centers


plant_br_data = {
    "Malt Plants": ["Afyon", "Konya", "Import (Izmir)"],
    "Istanbul": [0.026, 0.037, 0.032],
    "Ankara": [0.017, 0.017, 0.033],
    "Izmir": [0.020, 0.031, 0.004],
    "Sakarya": [0.019, 0.030, 0.028],
    "Adana": [0.032, 0.022, 0.048]
}

ship_plant_br = pd.DataFrame(plant_br_data)
ship_plant_br.set_index("Malt Plants", inplace=True)

# Data from the table
br_dc_data = {
    "Breweries": [
        "Istanbul", "Ankara", "Izmir", "Sakarya", "Adana"
    ],
    "Istanbul": [0.000, 0.032, 0.040, 0.011, 0.067],
    "Izmir": [0.040, 0.041, 0.000, 0.034, 0.064],
    "Antalya": [0.052, 0.039, 0.032, 0.041, 0.040],
    "Bursa": [0.017, 0.027, 0.023, 0.011, 0.060],
    "Kayseri": [0.055, 0.023, 0.062, 0.045, 0.024],
    "Export (Izmir)": [0.042, 0.043, 0.002, 0.036, 0.066]
}

# Create DataFrame
ship_br_dc = pd.DataFrame(br_dc_data)
ship_br_dc.set_index("Breweries", inplace=True)

# Data from the table
demand_data = {
    "Dist. Centers": [
        "Istanbul", "Izmir", "Antalya", "Bursa", "Kayseri", "Export (Izmir)", "Total"
    ],
    1: [103, 74, 50, 60, 102, 13, 402],
    2: [110, 80, 53, 75, 110, 13, 441],
    3: [125, 90, 60, 85, 125, 15, 500]
}

# Create DataFrame
demand = pd.DataFrame(demand_data)
demand.set_index("Dist. Centers", inplace=True)

malt_cap = {"Afyon": 30, "Konya": 68, "Import (Izmir)": 20}

V = {"Afyon": 8.333, "Konya": 8.333, "Import (Izmir)": 9.091}



br_cap_exist ={"Istanbul": 220, "Ankara": 200}

br_cap_pot_data = {
    "Potential Breweries": ["Izmir", "Sakarya", "Adana"],
    "New": [70, 70, 70],
    "Expansion": [50, 50, 50]
}
# Create DataFrame
br_cap_pot = pd.DataFrame(br_cap_pot_data)

In [71]:
m = gp.Model("Single Period Distribution")

x = m.addVars(plants, br_exist, lb=0.0, name='x')  # i --> j
y = m.addVars(br_exist, dc, lb=0.0, name='y') # j --> k

m.update()

In [72]:
# CONSTRAINTS

dc_demand = m.addConstrs((gp.quicksum(y[j, k] for j in br_exist) >= demand.loc[k, 1] for k in dc), name='Demand')


malt_capacity = m.addConstrs((gp.quicksum(x[i, j] for j in br_exist) <= malt_cap[i] for i in plants), name='Malt Capacity')

brew_capacity = m.addConstrs((gp.quicksum(x[i, j] * V[i] for i in plants) <= br_cap_exist[j] for j in br_exist), name='Brewery Capacity')

transshipment = m.addConstrs((gp.quicksum(x[i, j] * V[i] for i in plants) - gp.quicksum(y[j, k] for k in dc) >= 0 for j in br_exist), name='Transshipment')

m.update()

In [73]:
# OBJECTIVE

m.setObjective(gp.quicksum(ship_plant_br.loc[i, j] * x[i, j] for i in plants for j in br_exist) + gp.quicksum(ship_br_dc.loc[j, k] * y[j, k] for j in br_exist for k in dc), GRB.MINIMIZE)

# m.setObjective(0, GRB.MINIMIZE)

m.update()

In [74]:
m.optimize()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 7 5800HS with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 13 rows, 18 columns and 42 nonzeros
Model fingerprint: 0x8663a3a6
Coefficient statistics:
  Matrix range     [1e+00, 9e+00]
  Objective range  [2e-02, 6e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+02]
Presolve removed 1 rows and 0 columns
Presolve time: 0.01s
Presolved: 12 rows, 18 columns, 40 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   5.025000e+01   0.000000e+00      0s
      10    9.9082815e+00   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.02 seconds (0.00 work units)
Optimal objective  9.908281531e+00


In [75]:
for var in m.getVars():
    print(var.VarName, var.X)

x[Afyon,Istanbul] 24.24096963878555
x[Afyon,Ankara] 0.0
x[Konya,Istanbul] 0.0
x[Konya,Ankara] 24.000960038401537
x[Import (Izmir),Istanbul] 0.0
x[Import (Izmir),Ankara] 0.0
y[Istanbul,Istanbul] 103.0
y[Istanbul,Izmir] 39.0
y[Istanbul,Antalya] 0.0
y[Istanbul,Bursa] 60.0
y[Istanbul,Kayseri] 0.0
y[Istanbul,Export (Izmir)] 0.0
y[Ankara,Istanbul] 0.0
y[Ankara,Izmir] 35.0
y[Ankara,Antalya] 50.0
y[Ankara,Bursa] 0.0
y[Ankara,Kayseri] 102.0
y[Ankara,Export (Izmir)] 13.0


In [67]:
for cons in m.getConstrs():
    print(cons.ConstrName, m.getRow(cons).getValue(), cons.RHS)

Demand[Istanbul] 103.0 103.0
Demand[Izmir] 74.0 74.0
Demand[Antalya] 50.0 50.0
Demand[Bursa] 60.0 60.0
Demand[Kayseri] 102.0 102.0
Demand[Export (Izmir)] 13.0 13.0
Malt Capacity[Afyon] 24.24096963878555 30.0
Malt Capacity[Konya] 24.000960038401537 68.0
Malt Capacity[Import (Izmir)] 0.0 20.0
Brewery Capacity[Istanbul] 202.0 220.0
Brewery Capacity[Ankara] 200.0 200.0
Transshipment[Istanbul] 0.0 0.0
Transshipment[Ankara] 0.0 0.0
